## Create a function to upload files from s3 storage link

In [148]:
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd

- I would like to fetch and extract data files from the provided html link using requesets and beautifulsoup
- This will help the flow in the future unless partitions have changed

In [465]:
html_url = "https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/data-modeling.html"

r = requests.get(html_url)
soup = BeautifulSoup(r.content, 'html.parser')

In [466]:
parsed_html = soup

In [467]:
# found the file as a list we have about 3 files
amount_of_files = len(parsed_html.find_all('a'))

In [615]:
# looking how to parse individually for file name
data_files = parsed_html.find_all('a')[0]['href'].split('/')[-1].split('.')[0]
data_files

'receipts'

In [469]:
# parsing the actual link to upload the file
file_urls = parsed_html.find_all('a')[0]['href']
file_urls

In [616]:
data_files

'receipts'

In [ ]:
def parseHtml(html_url):
    
    r = requests.get(html_url)
    soup = BeautifulSoup(r.content, 'html.parser')
    parsed_html = soup
    amount_of_files = len(parsed_html.find_all('a'))
    

- After scraping the HTML I found that it was coming in as bytes
- We need to download the json.gz file and open it as a json file to read from pandas
- We also want delete the file after we turn it into a json file (no need for extra files)
- Last would be to get the json read as a pandas dataframe and save it as a csv file for further cleaning.

In [602]:
import wget
import tarfile
import shutil
import gzip
import os

In [628]:
# download the json.gz file
filename = wget.download(file_urls)

# unzip the gz file and write a new file as json
with gzip.open(filename) as g:
    with open(f'{data_files}.json', 'wb') as f_out:
        shutil.copyfileobj(g, f_out)

# if the gz file exists delete (save space) - or log for upload
if os.path.exists(f"{filename}"):
    os.remove(f"{filename}")
    print("old file removed ;)")
else:
    print("The file does not exist")

# if the file exist already we will rewrite it or we can log as new file if there is new date 
if os.path.exists(f"json_files/{data_files}.json"):
    shutil.copy(f'{data_files}.json', 'json_files/')
    print('File rewritten')
else:
# else the file does not exists we will move the file to the proper folder
    shutil.move(f'{data_files}.json', 'json_files/')
    print('File moved')


df = pd.read_json(f'json_files/{data_files}.json', lines = True)

path = 'csv_files/'
if not os.path.exists(path):
    os.makedirs(path)
    
df.to_csv(f'{path}{data_files}.csv')

old file removed ;)
File rewritten
